In [8]:
#Simulador TP 6

import random
import math
import numpy as np
import logging

#GENERAR ALEATORIAS
def valorRandom():
  r=random.uniform(0,1)
  return r

#def generarTiempoDeAtencion():
def generarIAFamiliar():
  r=random.uniform(0.15262557,0.95536)
  IAF=8.327+4.3297*math.tan(math.pi*(r-0.5))
  return IAF

def generarIANoFamiliar():
  r=random.uniform(0.025953,0.94262)
  IAI=14.99+1.2248*math.tan(math.pi*(r-0.5))
  return IAI

def generarTiempoDeAtencion():
  r=random.uniform(0.0159489,0.94225)
  TA=59.942+3.0059*math.tan(math.pi*(r-1/2))
  return TA

global contadorEsperaFamiliar
global contadorEsperaNoFamiliar
#EVENTOS
def llegadaFamiliar():
  global T, TF #Tiempo Y Tiempo final
  global TPLLF,TPLLI, TPSF,TPSI #Tiempos de llegada y salida
  global NTI,NTF,NI,NN, NM, NG #Cantidades totales de c/cliente, cantiades de c/cliente en el sist. y cantidades de c/mesas usadas
  global PPSF, SPSF,SPSI,PPSI #Permanecia en el sist de c/tipo y sus sumatorias
  global ITOF, ITOI, PTOI, PTOF #Porcentajes de tiempo ocioso de c/cliente (los q se sientan en la mesa sin poder pedir) y los inicios de tiempo ocioso
  global STOFi, STOFf #Sumatorias de tiempo ocioso
  global N, I, M, G #Variables de control
  global HV ,ARRF,ARRI #High value y cantidad de arrepentidos por tipo de cliente
  global vaciamiento
  global STOFf,STOFi
  global contadorEsperaFamiliar
  global contadorEsperaNoFamiliar
  global contadorEsperaFamiliar1
  global contadorEsperaNoFamiliar1
  global STOFf1,STOFi1
  
  #Permanecia en el sistema
  SPSF+=NG*(TPLLF-T)
  SPSI+=NI*(TPLLF-T)
  #Actualiza T
  T=TPLLF
  if vaciamiento is False:
    #Genera IAF
    IAF=generarIAFamiliar()
    #Actualiza TPLLF
    TPLLF=T+IAF
  r=valorRandom()
  #Arrepentimiento
  if(NG-M>5 and r>0.3):
    ARRF+=1
  else:
    #Se queda en el sistema
    NG+=1
    NTF+=1
    #Se fija si puede sentarse
    if(NM<M):
      NM+=1
      #busca en las mesas la que tenga tps=HV
      indiceMesaDisponible=buscarMesaDisponbleFamiliar(TPSF) 
      #Se fija si puede ser atendido
      if(NM<=3*G):
        #genera un tiempo de atencion TA
        TA=generarTiempoDeAtencion()
        #cambia el valor de tps de la mesa a T+TA
        TPSF[indiceMesaDisponible]=T+TA
      else:
        if(contadorEsperaFamiliar==0):
          STOFi+=T
        contadorEsperaFamiliar+=1
        STOFi1+=T
        contadorEsperaFamiliar1+=1

def salidaFamiliar():
  global T, TF #Tiempo Y Tiempo final
  global TPLLF,TPLLI, TPSF,TPSI #Tiempos de llegada y salida
  global NTI,NTF,NI,NN, NM, NG #Cantidades totales de c/cliente, cantiades de c/cliente en el sist. y cantidades de c/mesas usadas
  global PPSF, SPSF,SPSI,PPSI #Permanecia en el sist de c/tipo y sus sumatorias
  global ITOF, ITOI, PTOI, PTOF #Porcentajes de tiempo ocioso de c/cliente (los q se sientan en la mesa sin poder pedir) y los inicios de tiempo ocioso
  global STOFi, STOFf #Sumatorias de tiempo ocioso
  global N, I, M, G #Variables de control
  global HV ,ARRF,ARRI #High value y cantidad de arrepentidos por tipo de cliente
  global vaciamiento
  global STOFf,STOFi
  global contadorEsperaFamiliar
  global contadorEsperaNoFamiliar
  global contadorEsperaFamiliar1
  global contadorEsperaNoFamiliar1
  global STOFf1,STOFi1
  
  #busco la mesa a ser desocupada
  indiceMesaSalida=buscarMesaProximaASalir(TPSF)
  #Permanencia en el sistema
  SPSF+=NG*(TPSF[indiceMesaSalida]-T)
  SPSI+=NI*(TPSF[indiceMesaSalida]-T)
  #Actualizo T
  T=TPSF[indiceMesaSalida]
  #Actualizo el valor de tps de la mesa a HV
  TPSF[indiceMesaSalida]=HV
  #Desocupo mesa y se va cliente del sistema
  NG-=1
  NM-=1
  #Me fijo si el mozo desocupado puede atender un cliente que este sentado sin ser atendido, mirando la cantidad de mesas ocupadas
  if(NM>=3*G):
    #busca en las mesas la que tenga tps=HV
    indiceMesaDisponible=buscarMesaDisponbleFamiliar(TPSF)
    #termino el tiempo ocioso y lo sumo
    if(contadorEsperaFamiliar==1):
      STOFf+=T
    contadorEsperaFamiliar-=1
    STOFf1+=T
    ITOF[indiceMesaDisponible]=T
    #genera un tiempo de atencion TA
    TA=generarTiempoDeAtencion()
    #cambia el valor de tps de la mesa a T+TA
    TPSF[indiceMesaDisponible]=T+TA
    #Me fijo si, habiendo deocupado una mesa y mientras el mozo atiende a alguien, otro cliente puede sentarse
    if(NG>=M):
      NM+=1
      if(contadorEsperaFamiliar==0):
          STOFi+=T
      contadorEsperaFamiliar+=1
      STOFi1+=T
      contadorEsperaFamiliar1+=1
  #Me fijo si puedo atender a alguien que esta esperando mesa
  elif(NG>=M):
    NM+=1
    #genera un tiempo de atencion TA
    TA=generarTiempoDeAtencion()
    #cambia el valor de tps de la mesa a T+TA
    TPSF[indiceMesaSalida]=T+TA

def llegadaNoFamiliar():
  global T, TF #Tiempo Y Tiempo final
  global TPLLF,TPLLI, TPSF,TPSI #Tiempos de llegada y salida
  global NTI,NTF,NI,NN, NM, NG #Cantidades totales de c/cliente, cantiades de c/cliente en el sist. y cantidades de c/mesas usadas
  global PPSF, SPSF,SPSI,PPSI #Permanecia en el sist de c/tipo y sus sumatorias
  global ITOF, ITOI, PTOI, PTOF #Porcentajes de tiempo ocioso de c/cliente (los q se sientan en la mesa sin poder pedir) y los inicios de tiempo ocioso
  global STOFi, STOFf #Sumatorias de tiempo ocioso
  global STOIf,STOIi
  global N, I, M, G #Variables de control
  global HV ,ARRF,ARRI #High value y cantidad de arrepentidos por tipo de cliente
  global vaciamiento
  global STOFf,STOFi
  global contadorEsperaFamiliar
  global contadorEsperaNoFamiliar
  global contadorEsperaFamiliar1
  global contadorEsperaNoFamiliar1
  global STOIf1,STOIi1
  
  SPSF+=NG*(TPLLI-T)
  SPSI+=NI*(TPLLI-T)
  T=TPLLI
  if vaciamiento is False:
    IAI=generarIANoFamiliar()
    TPLLI=T+IAI
  r=valorRandom()
  if(NG-M>5 and r>0.3):
    ARRI+=1
  else:
    NI+=1
    NTI+=1
    if(NN<N):
      NN+=1
      #busca en las mesas la que tenga tps=HV
      indiceMesaDisponible=buscarMesaDisponbleNoFamiliar(TPSI) 
      if(NN<=3*I):
        #genera un tiempo de atencion TA
        TA=generarTiempoDeAtencion()
        #cambia el valor de tps de la mesa a T+TA
        TPSI[indiceMesaDisponible]=T+TA
      else:
        
        if(contadorEsperaNoFamiliar==0):
          STOIi+=T
        contadorEsperaNoFamiliar+=1
        STOIi1+=T
        contadorEsperaNoFamiliar1+=1

def salidaNoFamiliar():
  global T, TF #Tiempo Y Tiempo final
  global TPLLF,TPLLI, TPSF,TPSI #Tiempos de llegada y salida
  global NTI,NTF,NI,NN, NM, NG #Cantidades totales de c/cliente, cantiades de c/cliente en el sist. y cantidades de c/mesas usadas
  global PPSF, SPSF,SPSI,PPSI #Permanecia en el sist de c/tipo y sus sumatorias
  global ITOF, ITOI, PTOI, PTOF #Porcentajes de tiempo ocioso de c/cliente (los q se sientan en la mesa sin poder pedir) y los inicios de tiempo ocioso
  global STOFi, STOFf #Sumatorias de tiempo ocioso
  global STOIf,STOIi
  global N, I, M, G #Variables de control
  global HV ,ARRF,ARRI #High value y cantidad de arrepentidos por tipo de cliente
  global vaciamiento
  global STOFf,STOFi
  global contadorEsperaFamiliar
  global contadorEsperaNoFamiliar
  global contadorEsperaFamiliar1
  global contadorEsperaNoFamiliar1
  global STOIf1,STOIi1
  
  #busco la mesa a salir
  indiceMesaSalida=buscarMesaProximaASalir(TPSI)

  SPSF+=NG*(TPSI[indiceMesaSalida]-T)
  SPSI+=NI*(TPSI[indiceMesaSalida]-T)
  T=TPSI[indiceMesaSalida]
  TPSI[indiceMesaSalida]=HV
  NI-=1
  NN-=1
  if(NN>=3*I):
    #busca en las mesas la que tenga tps=HV
    indiceMesaDisponible=buscarMesaDisponbleNoFamiliar(TPSI)
    #termino el tiempo ocioso y lo sumo
    if(contadorEsperaNoFamiliar==1):
      STOIf+=T
    contadorEsperaNoFamiliar-=1
    STOIf1+=T
    ITOI[indiceMesaDisponible]=T
    #genera un tiempo de atencion TA
    TA=generarTiempoDeAtencion()
    #cambia el valor de tps de la mesa a T+TA
    TPSI[indiceMesaDisponible]=T+TA
    #Me fijo si, habiendo deocupado una mesa y mientras el mozo atiende a alguien, otro cliente puede sentarse
    if(NI>=N):
      NN+=1
      if(contadorEsperaNoFamiliar==0):
          STOIi+=T
      contadorEsperaNoFamiliar+=1
      STOIi1+=T
      contadorEsperaNoFamiliar1+=1
  elif(NI>=N):
    NN+=1
    #genera un tiempo de atencion TA
    TA=generarTiempoDeAtencion()
    #cambia el valor de tps de la mesa a T+TA
    TPSI[indiceMesaSalida]=T+TA


#FUNCIONES
def buscarMesaDisponbleNoFamiliar(tps):
  mesa=0
  for a in tps:
    indice=tps.index(a)
    if(a==HV and ITOI[indice]!=HV):
      mesa=indice
  return mesa

def buscarMesaDisponbleFamiliar(tps):
  mesa=0
  for a in tps:
    indice=tps.index(a)
    if(a==HV and ITOF[indice]!=HV):
      mesa=indice
  return mesa

def buscarMesaProximaASalir(tps):
  mesa=tps.index(min(tps))
  return mesa

def obtenerMeses(tiempo):
  meses=((tiempo/60)/24)/30
  return meses

def obtener_Proximo_Evento():
  global T, TF #Tiempo Y Tiempo final
  global TPLLF,TPLLI, TPSF,TPSI #Tiempos de llegada y salida
  global NTI,NTF,NI,NN, NM, NG #Cantidades totales de c/cliente, cantiades de c/cliente en el sist. y cantidades de c/mesas usadas
  global PPSF, SPSF,SPSI,PPSI #Permanecia en el sist de c/tipo y sus sumatorias
  global ITOF, ITOI, PTOI, PTOF #Porcentajes de tiempo ocioso de c/cliente (los q se sientan en la mesa sin poder pedir) y los inicios de tiempo ocioso
  global STOFi, STOFf #Sumatorias de tiempo ocioso
  global STOIf,STOIi
  global N, I, M, G #Variables de control
  global HV ,ARRF,ARRI #High value y cantidad de arrepentidos por tipo de cliente
  global contadorEsperaFamiliar1
  global contadorEsperaNoFamiliar1
  global STOIf1,STOIi1
  #Buscar el tiempo de salida de la prox mesa familiar que se va a desocupar
  indiceFamiliar=buscarMesaProximaASalir(TPSF)
  mesaFamiliarASalir=TPSF[indiceFamiliar]
  
  #Buscar el tiempo de salida de la prox mesa no familiar que se va a desocupar
  indiceNoFamiliar=buscarMesaProximaASalir(TPSI)
  mesaNoFamiliarASalir=TPSI[indiceNoFamiliar]
  
  #Pongo los tiempos de los proximos eventos en una lista y, despues, devuelvo el valor del indice de donde se encuentra el valor mas chico (el prox evento)
  eventos=[TPLLF,TPLLI,mesaFamiliarASalir,mesaNoFamiliarASalir]
  return eventos.index(min(eventos))


def calcularResultados():  
    print(f"SIMULACION PARA: \n {M} MESAS FAMILIARES \n {N} MESAS NO FAMILIARES \n {G}  MOZOS FAMILIARES \n {I} MOZOS NO FAMILIARES \n")
    print("===================================================\n")
    PPSF=SPSF/NTF
    PPSI=SPSI/NTI
    print(f"PERMANENCIA EN EL SISTEMA: PARA CLIENTES FAMILIARES={PPSF} Y PARA CLIENTES NO FAMILIARES={PPSI}\n")
    PTOF=STOFf-STOFi
    PTOI=STOIf-STOIi
    print(f"PORCENTAJES DE TIEMPO en el que hay clientes ESPERANDO EN MESA: clientes familiares {PTOF*100/(T*M)} y para clientes no familiares {PTOI*100/(T*N)}  \n")
    PAF=ARRF*100/(ARRF+NTF)
    PAI=ARRI*100/(ARRI+NTI)
    print(f"PORCENTAJES DE CLIENTES ARREPENTIDOS: PARA CLIENTES FAMILIARES={PAF} Y PARA CLIENTES NO FAMILIARES={PAI} Arrepentidos fam={ARRF} y no fam={ARRI}\n")
    mesesACobrar=obtenerMeses(T)
    COSTOMOZO=(I+G)*300000*mesesACobrar
    COSTOMESAIND=N*120000
    COSTOMESAGRANDE=M*300000
    print(f"COSTO TOTAL {COSTOMOZO+COSTOMESAGRANDE+COSTOMESAIND}: POR MOZOS={COSTOMOZO}, POR MESAS INDIVIDUALES={COSTOMESAIND} Y POR MESAS FAMILIARES={COSTOMESAGRANDE}")
    
#SIMULADOR
def simulacion(cantN,cantM,cantI,cantG):
  global T, TF #Tiempo Y Tiempo final
  global TPLLF,TPLLI, TPSF,TPSI #Tiempos de llegada y salida
  global NTI,NTF,NI,NN, NM, NG #Cantidades totales de c/cliente, cantiades de c/cliente en el sist. y cantidades de c/mesas usadas
  global PPSF, SPSF,SPSI,PPSI #Permanecia en el sist de c/tipo y sus sumatorias
  global ITOF, ITOI, PTO, PTOF #Porcentajes de tiempo ocioso de c/cliente (los q se sientan en la mesa sin poder pedir) y los inicios de tiempo ocioso
  global STOFi, STOFf #Sumatorias de tiempo ocioso
  global N, I, M, G #Variables de control
  global HV ,ARRF,ARRI #High value y cantidad de arrepentidos por tipo de cliente
  global vaciamiento
  global STOFf,STOFi
  global STOIf,STOIi
  global contadorEsperaFamiliar
  global contadorEsperaNoFamiliar
  global contadorEsperaFamiliar1
  global contadorEsperaNoFamiliar1
  global STOIf1,STOIi1
  global STOFf1,STOFi1

  vaciamiento=False

  contadorEsperaFamiliar1=0
  contadorEsperaNoFamiliar1=0
  contadorEsperaFamiliar=0
  contadorEsperaNoFamiliar=0
  #Sumatoria de permanencia en el sistema
  SPSF=0     
  SPSI=0
  #Arrepentidos
  ARRF=0 
  ARRI=0
  #High value
  HV=9999999999

  #Variables de Estado

  NN=0   #Cantidad de mesas individuales en uso
  NM=0   #Cantidad de mesas familiares en uso
  NG=0  #Cantidad clientes familiares en el sistema
  NI=0  #Cantidad clientes no familiares en el sistema
  NTI=0 #Cantidad clientes no familiares en total
  NTF=0 #Cantidad clientes familiares en total

  # Condiciones Iniciales

  T=0   #tiempo en minutos
  TF= 129600  #tiempo final en minutos
  TPLLF=0 #Tiempo de llegada de prox. cliente familiar
  TPLLI=0 #Tiempo de llegada de prox. cliente no familiar
  

  #Variables de Control
  N = cantN #cantidad de mesas no familiares
  M = cantM 
  I= cantI #cantidad de mozos no familiares
  G= cantG

  TPSF=[HV] * M  #Tiempo de Salida de MESAS FAMILIARES
  TPSI=[HV] * N  #Tiempo de salida de MESAS INDIVIDUALES

  #Variables de Resultados
  PPSF=0   #promedio de permanencia en el sistema de los clientes familiares
  PPSI=0   #promedio de permanencia en el sistema de los clientes no familiares
  PTO=0
  PTOF=0
  
  SPSI=0
  SPSF=0

  STOFi=0
  STOFf=0
  STOIi=0
  STOIf=0
  STOFi1=0
  STOFf1=0
  STOIi1=0
  STOIf1=0

  ITOF=[0]*M
  ITOI=[0]*N
  
  STOFi=0
  STOFf=0

  #print("Inicio: ", minutos_a_dias_horas_minutos(T))
 # print("Final: ",minutos_a_dias_horas_minutos(TF))

  #print("STOA: ", STOA)
  #print("ITOA: ", ITOA)

  while True:
    proximo_evento=obtener_Proximo_Evento()
    cases={
            0:llegadaFamiliar,
            1:llegadaNoFamiliar,
            2:salidaFamiliar,
            3:salidaNoFamiliar,
          }
    cases.get(proximo_evento)()
    #print(f"T={T} cf={NG} ci={NI} mf={NM} mi={NN}\n tpsf={TPSF} \n tpsi={TPSI} \n tpllf={TPLLF}\n tplli={TPLLI}")
    #print(contadorEsperaFamiliar)
    if (T<=TF):
      continue
    elif(NI+NG==0):
      break
    else:
      TPLLI=HV
      TPLLF=HV
      vaciamiento=True
      continue
  calcularResultados()
  


In [14]:
simulacion(15,10,5,3)

SIMULACION PARA: 
 10 MESAS FAMILIARES 
 15 MESAS NO FAMILIARES 
 3  MOZOS FAMILIARES 
 5 MOZOS NO FAMILIARES 


PERMANENCIA EN EL SISTEMA: PARA CLIENTES FAMILIARES=58.700165636539765 Y PARA CLIENTES NO FAMILIARES=58.74741871900271

PORCENTAJES DE TIEMPO en el que hay clientes ESPERANDO EN MESA: clientes familiares 0.1227078930498846 y para clientes no familiares 0.0  0.0 Y 1.2270789304988459

PORCENTAJES DE CLIENTES ARREPENTIDOS: PARA CLIENTES FAMILIARES=0.0 Y PARA CLIENTES NO FAMILIARES=0.0 Arrepentidos fam=0 y no fam=0

COSTO TOTAL 12003394.333378905: POR MOZOS=7203394.333378905, POR MESAS INDIVIDUALES=1800000 Y POR MESAS FAMILIARES=3000000
